## Зачем нужен слой эмбединга?

Данный слой связан с парадигмой Word2Vec (Векторное представление слов). Как различить слова  "идти" и "шагать"?  Для компьютера это разные строки, но по векторному преставлению они будут иметь близкие векторы.

Этот слой необходим, потому что многие алгоритмы машинного обучения (включая глубокие сети) требуют, чтобы их входные данные были векторами непрерывных значений; Они не будут работать со строками простого текста. Поэтому данный слой используется для сопоставления слов или фраз из словаря с соответствующим вектором действительных чисел. 

## Чем отличается RNN от LSTM?

Все RNN имеют обратные связи в рекуррентном слое. Это позволяет им сохранять информацию в памяти с течением времени. Но может быть сложно обучить стандартные RNN для решения проблем, требующих изучения долгосрочных временных зависимостей. Это связано с тем, что градиент функции потерь экспоненциально убывает со временем (проблема исчезающего градиента). Сети LSTM - это тип RNN, который использует специальные модули в дополнение к стандартным. Блоки LSTM включают в себя «ячейку памяти», которая может хранить информацию в памяти в течение длительных периодов времени. Набор "вентилей" используется для контроля того, когда информация поступает в память, когда она выводится, и когда она забывается. Эта архитектура позволяет лучше управлять потоком градиента и обеспечивают лучшее сохранение «долгосрочных зависимостей». 

## Чем RNN лучше чем свёрточная сеть?

Сверточные нейронные сети (CNN) предназначены для обработки изображений. Рекуррентные нейронные сети (RNN) предназначены для обработки речевого сигнала или текста.

CNN принимают вход фиксированного размера и генерируют выходы фиксированного размера. RNN, с другой стороны, может обрабатывать произвольные длины ввода / вывода.

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [28]:
df = pd.read_csv("/Users/Anton/Desktop/Учеба/Тексты/hw3/avito_train_1kk.tsv",sep='\t')

In [29]:
print ("Blocked ratio",df.is_blocked.mean())
print ("Count:",len(df))

Blocked ratio 0.22822210732570425
Count: 1204949


## Балансировка выборки

In [30]:
#Балансировка выборки таким образом, чтобы  0.49 < blocked ratio < 0.51 и размер df <= 560k
df = df[df.is_blocked == 1].append(df[df.is_blocked == 0].sample(285000))

In [31]:
print ("Blocked ratio",df.is_blocked.mean())
print ("Count:",len(df))

Blocked ratio 0.491067793341381
Count: 559996


In [32]:
#Удовлетворяем требованиям
assert df.is_blocked.mean() < 0.51
assert df.is_blocked.mean() > 0.49
assert len(df) <= 560000

print ("All tests passed")

All tests passed


## Токенизация

In [33]:
from nltk.tokenize import RegexpTokenizer
from collections import Counter,defaultdict
tokenizer = RegexpTokenizer(r"\w+")

#Dictionary of tokens
token_counts = Counter()

#All texts
all_texts = np.hstack([df.description.values,df.title.values])


#Compute token frequencies
for s in all_texts:
    if type(s) is not str:
        continue
    tokens = tokenizer.tokenize(s)
    for token in tokens:
        token_counts[token] +=1

## Удаляем редкие токены

In [34]:
min_count = 10
tokens = [token for token in token_counts.keys() if token_counts[token] >= 10]

In [35]:
token_to_id = {t:i+1 for i,t in enumerate(tokens)}
null_token = "NULL"
token_to_id[null_token] = 0

In [36]:
print ("# Tokens:",len(token_to_id))
if len(token_to_id) < 30000:
    print ("Alarm! It seems like there are too few tokens. Make sure you updated NLTK and applied correct thresholds -- unless you now what you're doing, ofc")
if len(token_to_id) > 1000000:
    print ("Alarm! Too many tokens. You might have messed up when pruning rare ones -- unless you know what you're doin' ofc")

# Tokens: 106642


## Меняем тексты и ID

In [37]:
def vectorize(strings, token_to_id, max_len=150):
    token_matrix = []
    for s in strings:
        if type(s) is not str:
            token_matrix.append([0]*max_len)
            continue
        tokens = tokenizer.tokenize(s)
        token_ids = list(map(lambda token: token_to_id.get(token,0), tokens))[:max_len]
        token_ids += [0]*(max_len - len(token_ids))
        token_matrix.append(token_ids)

    return np.array(token_matrix)

In [38]:
desc_tokens = vectorize(df.description.values,token_to_id,max_len = 150)
title_tokens = vectorize(df.title.values,token_to_id,max_len = 15)

In [39]:
# Например
print ("Размер матрицы:",title_tokens.shape)
for title, tokens in zip(df.title.values[:3],title_tokens[:3]):
    print (title,'->', tokens[:10],'...')

Размер матрицы: (559996, 15)
Поездки на таможню, печать в паспорте -> [    1     2     3 21589    74  8657     0     0     0     0] ...
Рефлекторно-урогинекологический массаж -> [15548     0   363     0     0     0     0     0     0     0] ...
Возьму суду под200 т. р -> [ 62  63   0 146 358   0   0   0   0   0] ...


## Предварительная обработка некоторых данных

In [40]:
df_numerical_features = df[["phones_cnt","emails_cnt","urls_cnt","price"]]

In [41]:
from sklearn.feature_extraction import DictVectorizer

categories = []
data_cat_subcat = df[["category","subcategory"]].values

categories = [{"category":category_name, "subcategory":subcategory_name}\
              for category_name, subcategory_name in data_cat_subcat]

vectorizer = DictVectorizer(sparse=False)
cat_one_hot = vectorizer.fit_transform(categories)
cat_one_hot = pd.DataFrame(cat_one_hot,columns=vectorizer.feature_names_)

In [42]:
df_non_text = pd.merge(
    df_numerical_features,cat_one_hot,on = np.arange(len(cat_one_hot))
)
del df_non_text["key_0"]

## Разделение выборки

In [43]:
#Target variable - whether or not sample contains prohibited material
target = df.is_blocked.values.astype('int32')
#Preprocessed titles
title_tokens = title_tokens.astype('int32')
#Preprocessed tokens
desc_tokens = desc_tokens.astype('int32')
#Non-sequences
df_non_text = df_non_text.values.astype('float32')

In [44]:
from sklearn.cross_validation import train_test_split
splits = train_test_split(title_tokens,desc_tokens,df_non_text,target,test_size = 0.3)

title_tr,title_ts,desc_tr,desc_ts,nontext_tr,nontext_ts,target_tr,target_ts = splits


/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Сохранение предобработанных данных

In [5]:
save_prepared_data = False #save
read_prepared_data = True #load

#but not both at once
assert not (save_prepared_data and read_prepared_data)

if save_prepared_data:
    print ("Saving preprocessed data (may take up to 3 minutes)")

    import pickle
    data_tuple = title_tr,title_ts,desc_tr,desc_ts,nontext_tr,nontext_ts,target_tr,target_ts
    with open("preprocessed_data.pcl",'wb') as fout:
        pickle.dump(data_tuple,fout)
    with open("token_to_id.pcl",'wb') as fout:
        pickle.dump(token_to_id,fout)

    print ("готово")
    
elif read_prepared_data:
    print ("Reading saved data...")
    
    import pickle
    
    with open("preprocessed_data.pcl",'rb') as fin:
        data_tuple = pickle.load(fin)
    title_tr,title_ts,desc_tr,desc_ts,nontext_tr,nontext_ts,target_tr,target_ts = data_tuple
    with open("token_to_id.pcl",'rb') as fin:
        token_to_id = pickle.load(fin)
        
    #Re-importing libraries to allow staring noteboook from here
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    %matplotlib inline
   
    print ("done")

Reading saved data...
done


## Обучение

In [4]:
! pip install --upgrade https://github.com/Theano/Theano/archive/master.zip
! pip install --upgrade https://github.com/Lasagne/Lasagne/archive/master.zip

    100% |████████████████████████████████| 13.2MB 45kB/s eta 0:00:011   34% |███████████▏                    | 4.6MB 1.5MB/s eta 0:00:06
    100% |████████████████████████████████| 4.4MB 134kB/s ta 0:00:011    86% |███████████████████████████▉    | 3.9MB 8.3MB/s eta 0:00:01
    100% |████████████████████████████████| 16.2MB 35kB/s eta 0:00:011
Requirement already up-to-date: six>=1.9.0 in /anaconda/lib/python3.6/site-packages (from Theano==0.9.0)
  Found existing installation: numpy 1.11.3
    Uninstalling numpy-1.11.3:
      Successfully uninstalled numpy-1.11.3
  Found existing installation: scipy 0.18.1
    Uninstalling scipy-0.18.1:
      Successfully uninstalled scipy-0.18.1
  Running setup.py install for Theano ... - \ | / - \ | / - \ | / - \ | / - \ | / - done
    100% |████████████████████████████████| 225kB 495kB/s ta 0:00:01
Requirement already up-to-date: numpy in /anaconda/lib/python3.6/site-packages (from Lasagne==0.2.dev1)
  Runn

In [6]:
#libraries
import lasagne
from theano import tensor as T
import theano

In [7]:
#3 inputs and a refere output
title_token_ids = T.matrix("title_token_ids",dtype='int32')
desc_token_ids = T.matrix("desc_token_ids",dtype='int32')
categories = T.matrix("categories",dtype='float32')
target_y = T.ivector("is_blocked")

## Архитектура сети

In [8]:
title_inp = lasagne.layers.InputLayer((None,title_tr.shape[1]),input_var=title_token_ids)
descr_inp = lasagne.layers.InputLayer((None,desc_tr.shape[1]),input_var=desc_token_ids)
cat_inp = lasagne.layers.InputLayer((None,nontext_tr.shape[1]), input_var=categories)

In [9]:
# Descriptions
#word-wise embedding. We recommend to start from some 64 and improving after you are certain it works.
descr_nn = lasagne.layers.EmbeddingLayer(descr_inp, input_size=len(token_to_id)+1, output_size=128)
descr_nn = lasagne.layers.LSTMLayer(descr_nn, 128,
                                    grad_clipping=100, nonlinearity=lasagne.nonlinearities.tanh, only_return_final=True)

# Titles
title_nn = lasagne.layers.EmbeddingLayer(title_inp, input_size=len(token_to_id)+1, output_size=128)
title_nn = lasagne.layers.LSTMLayer(title_nn, 128,
                                    grad_clipping=100, nonlinearity=lasagne.nonlinearities.tanh, only_return_final=True)

# Non-sequences
cat_nn = lasagne.layers.DenseLayer(cat_inp, 64)
cat_nn = lasagne.layers.DenseLayer(cat_nn, 32)

In [10]:
nn = lasagne.layers.concat([descr_nn, title_nn, cat_nn])

nn = lasagne.layers.DenseLayer(nn, 512)
nn = lasagne.layers.DropoutLayer(nn, p=0.3)
nn = lasagne.layers.DenseLayer(nn, 1, nonlinearity=lasagne.nonlinearities.linear)

## Функция потерь

In [11]:
#All trainable params
weights = lasagne.layers.get_all_params(nn,trainable=True)

In [12]:
#Simple NN prediction
prediction = lasagne.layers.get_output(nn)[:,0]

#Hinge loss
loss = lasagne.objectives.binary_hinge_loss(prediction,target_y,delta = 1., log_odds=True).mean()

In [13]:
#Weight optimization step
updates = lasagne.updates.adamax(loss, weights)

## Прогнозирование

In [14]:
#deterministic version
det_prediction = lasagne.layers.get_output(nn,deterministic=True)[:,0]
#equivalent loss function
det_loss = lasagne.objectives.binary_hinge_loss(det_prediction,target_y,delta = 1., log_odds=True).mean()

In [15]:
train_fun = theano.function([desc_token_ids,title_token_ids,categories,target_y],[loss,prediction],updates = updates)
eval_fun = theano.function([desc_token_ids,title_token_ids,categories,target_y],[det_loss,det_prediction])

/anaconda/lib/python3.6/site-packages/theano/tensor/basic.py:5130: UserWarning: flatten outdim parameter is deprecated, use ndim instead.
  "flatten outdim parameter is deprecated, use ndim instead.")


## Цикл обучения

In [25]:
#average precision at K

def APatK ( y_true,y_predicted, K =32500):
    """Calculates AP@k given true Y and predictions (probabilities).
    Sorts answers by y_predicted to obtain ranking"""

    sort_by_ypred = np.argsort(-y_predicted)
    
    y_true = y_true[sort_by_ypred]
    y_predicted = y_predicted[sort_by_ypred]
    
    countRelevants = 0
    listOfPrecisions = []
    
    for i in range(min(K,len(y_true))):
        currentk = i + 1.0
        if y_true[i] !=0:
            countRelevants+=1
        precisionAtK = countRelevants / currentk 
        listOfPrecisions.append(precisionAtK)
    return np.sum( listOfPrecisions ) / min(K,len(y_true)) 

def score(final_accuracy,final_auc,final_apatk):
    
    print ("\nAUC:")
    if final_auc >= 0.99:
        print ("\tПиши статью. (great)")
    elif final_auc >= 0.97:
        print ("\tОтличное решение! (good)")
    elif final_auc >= 0.95:
        print ("\tСойдёт, хотя можно ещё поднажать (ok)")
    elif final_auc >= 0.9:
        print ("\tНеплохо, но ты можешь лучше! (not ok)")
    elif final_auc > 0.8:
        print ("\tТы на правильном пути! (not ok)")
    elif final_auc > 0.65:
        print ("\tДобавь жару! (not ok)")
    else:
        print ("\tМожет быть, она недоучилась? Ну или слишком маленькая? Или в детстве болела? (not ok)")
    
        
    print ("\nAccuracy:")
    if final_accuracy >= 0.97:
        print ("\tОчешуенно! (great)")
    elif final_accuracy >= 0.95:
        print ("\tОтличный результат! (good)")
    elif final_accuracy >= 0.9:
        print ("\tВсё ок (ok)")
    else:
        print ("Надо бы подтянуть. (not ok)")

    print ("\nAverage precision at K:")
    if final_apatk > 0.99:
        print ("\tЗасабмить на kaggle! (great) \n\t Нет, ну честно - выкачай avito_test.tsv, засабмить и скажи, что вышло.")
    elif final_apatk > 0.95:
        print ("\tОтличный результат (good)")
    elif final_apatk > 0.92:
        print ("\tВы побили baseline (ok)")
    else:
        print ("\tНадо бы поднажать (not ok)")
        


In [17]:
# Out good old minibatch iterator now supports arbitrary amount of arrays (X,y,z)

def iterate_minibatches(*arrays,**kwargs):
    batchsize=kwargs.get("batchsize",100)
    shuffle = kwargs.get("shuffle",True)
    
    if shuffle:
        indices = np.arange(len(arrays[0]))
        np.random.shuffle(indices)
    for start_idx in range(0, len(arrays[0]) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield [arr[excerpt] for arr in arrays]

Тренил на 50 эпохах и остановил процесс, когда получился достаточно хороший скор.

In [22]:
from sklearn.metrics import roc_auc_score, accuracy_score

n_epochs = 50
batch_size = 100
minibatches_per_epoch = 100

for i in range(n_epochs):
    #training
    epoch_y_true = []
    epoch_y_pred = []
    print ("Epoch: " + str(i))
    b_c = b_loss = 0
    for j, (b_desc,b_title,b_cat, b_y) in enumerate(
        iterate_minibatches(desc_tr,title_tr,nontext_tr,target_tr,batchsize=batch_size,shuffle=True)):
        if j > minibatches_per_epoch:break
            
        loss,pred_probas = train_fun(b_desc,b_title,b_cat,b_y)
        
        b_loss += loss
        b_c +=1
        
        epoch_y_true.append(b_y)
        epoch_y_pred.append(pred_probas)
    
    epoch_y_true = np.concatenate(epoch_y_true)
    epoch_y_pred = np.concatenate(epoch_y_pred)
    
    print ("Train:")
    print ('\tloss:',b_loss/b_c)
    print ('\tacc:',accuracy_score(epoch_y_true,epoch_y_pred>0.))
    print ('\tauc:',roc_auc_score(epoch_y_true,epoch_y_pred))
    print ('\tap@k:',APatK(epoch_y_true,epoch_y_pred,K = int(len(epoch_y_pred)*0.025)+1))
    
    #evaluation
    epoch_y_true = []
    epoch_y_pred = []
    b_c = b_loss = 0
    for j, (b_desc,b_title,b_cat, b_y) in enumerate(
        iterate_minibatches(desc_ts,title_ts,nontext_tr,target_ts,batchsize=batch_size,shuffle=True)):
        if j > minibatches_per_epoch: break
        loss,pred_probas = eval_fun(b_desc,b_title,b_cat,b_y)
        
        b_loss += loss
        b_c +=1
        
        epoch_y_true.append(b_y)
        epoch_y_pred.append(pred_probas)

    epoch_y_true = np.concatenate(epoch_y_true)
    epoch_y_pred = np.concatenate(epoch_y_pred)
    
    print ("Val:")
    print ('\tloss:',b_loss/b_c)
    print ('\tacc:',accuracy_score(epoch_y_true,epoch_y_pred>0.))
    print ('\tauc:',roc_auc_score(epoch_y_true,epoch_y_pred))
    print ('\tap@k:',APatK(epoch_y_true,epoch_y_pred,K = int(len(epoch_y_pred)*0.025)+1))

Epoch: 0
Train:
	loss: 0.15718284054
	acc: 0.936534653465
	auc: 0.97924985044
	ap@k: 0.996609209414
Val:
	loss: 0.194513397073
	acc: 0.919405940594
	auc: 0.975087601206
	ap@k: 1.0


## Финальный подсчет

In [26]:
#evaluation
epoch_y_true = []
epoch_y_pred = []

b_c = b_loss = 0
for j, (b_desc,b_title,b_cat, b_y) in enumerate(
    iterate_minibatches(desc_ts,title_ts,nontext_tr,target_ts,batchsize=batch_size,shuffle=True)):
    loss,pred_probas = eval_fun(b_desc,b_title,b_cat,b_y)

    b_loss += loss
    b_c +=1

    epoch_y_true.append(b_y)
    epoch_y_pred.append(pred_probas)


epoch_y_true = np.concatenate(epoch_y_true)
epoch_y_pred = np.concatenate(epoch_y_pred)

final_accuracy = accuracy_score(epoch_y_true,epoch_y_pred>0)
final_auc = roc_auc_score(epoch_y_true,epoch_y_pred)
final_apatk = APatK(epoch_y_true,epoch_y_pred,K = int(len(epoch_y_pred)*0.025)+1)

print ("Scores:")
print ('\tloss:',b_loss/b_c)
print ('\tacc:',final_accuracy)
print ('\tauc:',final_auc)
print ('\tap@k:',final_apatk)
score(final_accuracy,final_auc,final_apatk)

Scores:
	loss: 0.202780129969
	acc: 0.91724836212
	auc: 0.973360833128
	ap@k: 0.993394280157

AUC:
	Отличное решение! (good)

Accuracy:
	Всё ок (ok)

Average precision at K:
	Засабмить на kaggle! (great) 
	 Нет, ну честно - выкачай avito_test.tsv, засабмить и скажи, что вышло.
